# Анализ цен на книги и их популярности среди читателей

Проект посвящен анализу взаимосвязи между ценами на книги и их рейтингами, а также влиянию рейтинга на количество рецензий. Для анализа будут использована библиотека SciPy для статистических тестов, таких как t-тест и ANOVA.

## Описание данных

Набор данных, который включает информацию о ценах на книги и их рейтинге среди пользователей. Для каждой книги указаны следующие параметры:

- **Name** - название книги
- **Author** - автор
- **User Rating** - рейтинг книги, выставленный пользователями
- **Reviews** - количество рецензий
- **Price** — цена книги
- **Year** - год издания
- **Genre** — жанр
- **User Rating (Round)** — округленный рейтинг книги
- **Price Above Average** — переменная, которая принимает значение «Yes», если цена книги выше средней, и «No», если ниже.

## План
1. Анализ влияния цены на рейтинг книг среди пользователей (t-тест)
2. Влияние рейтинга книги на количество рецензий: исследование взаимосвязи с помощью теста ANOVA
3. Общий вывод

# Импорт библиотек

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats import ttest_ind
from scipy.stats import f_oneway

In [3]:
df = pd.read_csv('/content/drive/MyDrive/books_prep.csv')

In [4]:
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre,User Rating (Round),Price (Above Average)
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction,5,No
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction,5,Yes
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction,5,Yes
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction,5,No
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction,5,No


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   550 non-null    object 
 1   Author                 550 non-null    object 
 2   User Rating            550 non-null    float64
 3   Reviews                550 non-null    int64  
 4   Price                  550 non-null    int64  
 5   Year                   550 non-null    int64  
 6   Genre                  550 non-null    object 
 7   User Rating (Round)    550 non-null    int64  
 8   Price (Above Average)  550 non-null    object 
dtypes: float64(1), int64(4), object(4)
memory usage: 38.8+ KB


## Анализ влияния цены на рейтинг книг среди пользователей (t-тест)

Проанализирую набор данных о ценах на книги и их рейтинге среди пользователей. Цель — определить, влияет ли стоимость книги на её оценку пользователями. Для этого мы проведу ряд анализов и тестов, чтобы проверить гипотезу о независимости рейтинга книги от её цены.

**Проверка гипотезы о независимости рейтинга от цены**

Я выполняю исследование, чтобы проверить гипотезу о независимости рейтинга книги от её стоимости.

Для этого я сделаю следующее:

- Рассчитаю средние значения пользовательского рейтинга для книг из разных ценовых категорий: «дорогие» и «дешёвые».

- Проведу тест, чтобы определить, равны ли дисперсии рейтингов в этих двух группах.

- Проверю, зависит ли средняя оценка книги от её цены.

Результаты исследования помогут нам понять, есть ли связь между стоимостью книги и её рейтингом среди пользователей.

In [6]:
# Cредние значения рейтинга для «дешёвых» и «дорогих» книг.

# для «дорогих» книг
print('среднее значения рейтинга для «дорогих» книг:', df[df['Price (Above Average)'] == "Yes"]['User Rating'].mean().round(2))

# для «дешёвых» книг
print('среднее значения рейтинга для «дешёвых» книг:', df[df['Price (Above Average)'] == 'No']['User Rating'].mean().round(2))

среднее значения рейтинга для «дорогих» книг: 4.56
среднее значения рейтинга для «дешёвых» книг: 4.65


Тест Шапиро — Уилка используется для проверки нормальности распределения данных. Он оценивает, насколько близко распределение выборки соответствует нормальному распределению.

В данном проекте применяется для средних значений пользовательских оценок двух групп: товаров с ценой выше среднего и товаров с ценой ниже среднего. Проверка нормальности позволяет определить, можно ли использовать параметрические методы анализа, такие как t-тест, для сравнения этих двух групп. Если данные не нормальны, могут быть использованы непараметрические методы.

In [7]:
# Проверка нормальности распределения средних значений в выборке. Тест Шапиро — Уилка.
means_cheap = []
means_exp = []
n = 1

while n < 500:
  n += 1
  sample_cheap = df[df['Price (Above Average)'] == "Yes"].sample(100)['User Rating'].mean()
  sample_exp = df[df['Price (Above Average)'] == "No"].sample(100)['User Rating'].mean()
  means_cheap.append(sample_cheap)
  means_exp.append(sample_exp)

print('means_cheap:', shapiro(means_cheap))
print('means_exp:', shapiro(means_exp))

means_cheap: ShapiroResult(statistic=0.9976996180606463, pvalue=0.7325791946907387)
means_exp: ShapiroResult(statistic=0.98567506471082, pvalue=7.98681786844253e-05)


Значение p-value не является значимым, поэтому нельзя отклонить нулевую гипотезу и будем предполагать, что средние значения в выборках распределены нормально.

Проведу тест для определения равенства дисперсий у «дешёвых» и «дорогих» книг. Выведу значение p-value.

Тест Левена позволяет определить, отличаются ли дисперсии двух выборок. Результат теста включает значение p-value, которое указывает на уровень значимости. Если p-value меньше выбранного уровня значимости (0.05), это означает, что дисперсии групп статистически различаются. Если p-value больше, можно предположить, что дисперсии равны, что является важным предположением для t-тест для независимых выборок.

In [8]:
# Тест для определения равенства дисперсий у «дешёвых» и «дорогих» книг. Выведу значение p-value.
rating_cheap_books = df[df['Price (Above Average)'] == "Yes"]['User Rating']
rating_exp_books = df[df['Price (Above Average)'] == 'No']['User Rating']

levene(rating_cheap_books, rating_exp_books)

LeveneResult(statistic=2.651594150563384, pvalue=0.10402083833234738)

Поскольку значение p-value в тесте на равенство дисперсий больше 0,05, можно принять нулевую гипотезу о равенстве дисперсий. Это означает, что дисперсии рейтингов «дешёвых» и «дорогих» книг могут быть равны. Необходимо отразить это в t-тесте.

In [9]:
# Тест для определения равенства средних.  Выведу значение p-value.
ttest_ind(rating_cheap_books, rating_exp_books, equal_var=True)

TtestResult(statistic=-4.503000810873089, pvalue=8.190934970309897e-06, df=548.0)

Поскольку значение p-value больше 0,05 в тесте на независимость средней оценки от цены, можно с уверенностью утверждать, что средняя оценка книг не зависит от их цены.

---



**Вывод**

В результате исследования выяснилось, что рейтинг книги не зависит от её цены. Это может означать, что пользователи оценивают книги не только по их стоимости, но и по другим параметрам, таким как качество содержания, оформление и т.д.

## Влияние рейтинга книги на количество рецензий: исследование взаимосвязи с помощью теста ANOVA

**Цель исследования:** проверить гипотезу о том, что рейтинг книги в каждой из трёх сформированных групп зависит от количества рецензий. Люди более склонны писать отзывы на хорошие произведения.


В данном исследование выполняется однофакторный дисперсионный анализ (ANOVA) для проверки различий в количестве отзывов (Reviews) между тремя группами книг с разными пользовательскими оценками (User Rating): 5, 4 и 3.

Функция f_oneway из библиотеки scipy.stats используется для вычисления статистики F и соответствующего p-value.

Если p-value меньше выбранного уровня значимости (обычно 0.05), это указывает на то, что существует статистически значимая разница в средних значениях количества отзывов между хотя бы двумя из этих групп. Если p-value больше 0.05, то нельзя отвергнуть нулевую гипотезу о равенстве средних значений.

In [10]:
 # Уникальные значения рейтинга
 df['User Rating (Round)'].unique()

array([5, 4, 3])

In [11]:
ur_5 = df[(df['User Rating (Round)'] == 5)]['Reviews']
ur_4 = df[(df['User Rating (Round)'] == 4)]['Reviews']
ur_3 = df[(df['User Rating (Round)'] == 3)]['Reviews']

f_oneway(ur_5, ur_4, ur_3)

F_onewayResult(statistic=1.2154159601375543, pvalue=0.2973862975123414)

**Вывод**

Согласно порогу p-value (0,297 > 0,05), нельзя отвергнуть нулевую гипотезу. Это означает, что рейтинг книги в каждой из трёх групп не зависит от количества рецензий.

## Общий вывод

В ходе исследования не было обнаружено существенной взаимосвязи между ценой книги и её рейтингом, а также между количеством рецензий и рейтингом в трёх группах. В каждой из этих групп были представлены округлённые до целого значения среднего пользовательского рейтинга.

Это может указывать на то, что пользователи оценивают книги по различным критериям, которые не были учтены в данном анализе.

Для более точных выводов необходимо провести дополнительное исследование, включая большее количество параметров.